# Technical Appendix: Structural Risk Model

下面我按 Chapter 3 的 Technical Appendix（3A） 的结构，把它“从假设→估计→用法→归因”的整套逻辑讲透

## 怎么选风险因子？

在 G&K 的语境下，一个“能进入风险模型的因子”至少要满足：
 - 能形成稳定的暴露 $X$（可观测、可解释、可标准化、可跨资产比较） ￼
 - 其因子收益$b$可由横截面回归稳健识别（不会被严重共线性/噪声淹没；行业因子因正交性强通常没问题） ￼
 - 它对应的 factor portfolio 真的在市场里体现为“共同波动维度”：也就是许多资产在该维度上同向振动，从而解释相关性（风险模型的目的） ￼
 - 可预测其风险：最终你要能给$F$一个“对未来的预测”，让模型能用于前瞻风险控制。

risk indices（size/value/momentum/volatility…）之所以是好风险因子，是因为它们通常能构造出跨行业的“单位暴露组合”，并且这些组合的收益/波动在历史上确实表现出稳定的共同结构（因此能解释协方差）。

## 风险模型先立“生成式”：回报是“共同因子 + 特异项”

Technical Appendix 一上来把风险模型写成
$$
r = X b + u
$$
 - $r: N$ 只股票的超额收益向量
 - $X: N\times K$ 的因子暴露矩阵（列是因子，行是股票）
 - 这里的因子包括：行业因子 + risk indices 因子（如 size/value/momentum/volatility…）
 - $b: K$ 个因子的当期因子收益（因子回报）
 - $u$：每只股票的特异收益（specific / idiosyncratic）

接着给两个关键假设：

:::{important}
特异收益与因子收益不相关
$$
\mathrm{Cov}(u_n, b_k)=0
$$
这让“共同因子风险”和“特异风险”能干净拆开。
:::

:::{important}
不同股票的特异收益互不相关
$$
\mathrm{Cov}(u_n, u_m)=0, (m\neq n)
$$
也就是 A（特异协方差）通常取对角。 ￼
:::

于是得到协方差结构（这就是结构性风险模型）：
$$
V = X F X' + A
$$
 - $F$：因子收益$b$的$K\times K$协方差（因子协方差）
 - $A$：特异方差的对角矩阵（specific variance） ￼

这三件套$(X,F,A)$就是风险模型：它把 “相关性从哪里来” 归结为 共同暴露$X$通过 因子协方差$F$传播出来；剩下的都是对角的$A$。

## 因子暴露矩阵$X$是可观测的数据

$X$的列就是risk factors/indices

在第三章正文里，他们把 common factors 分为两类：industries 和 risk indices。risk indices 是“横跨行业的系统性维度”（风格/特征维度），例如 volatility、momentum、size、liquidity、growth、value、earnings volatility、financial leverage 等。 ￼

关键是：risk index 不是某一个原始量，而是若干 descriptors（描述子）加权组合：同一类里放多个相关但不完全相同的测量（比如 volatility 可以用历史波动、隐含波动、beta、价格区间等）。他们强调用加权方式来最大化模型的解释/预测能力并提升稳健性。 ￼

为了让不同量纲可比，descriptor 暴露先做标准化：
$$
X_{\text{normalized}}=\frac{X_{\text{raw}}-\mathrm{mean}[X_{\text{raw}}]}{\mathrm{Std}[X_{\text{raw}}]}
$$
标准化后每个暴露在横截面上均值 0、标准差 1，便于组合与处理极端值。 ￼

所以 risk indices 在 X 里就是一些列：
 - 行业列通常像“截距/分组”一样解释横截面的平均差异；
 - risk index 列捕捉“行业之外的共同波动结构”。 ￼

## 统计估计方法

### 怎么从数据里把$b$（因子收益）抠出来？

因为 (3A.1) 就是横截面回归，做法是每期做一次 Fama–MacBeth 式回归：用当期股票收益 $r$ 去回归暴露 $X$，得到当期因子收益 $\hat b$。

他们强调用 GLS 加权回归：每个股票的观测按其特异方差的倒数加权（特异方差小的股票更“干净”，权重更大）：
$$
\hat b = (X' A^{-1} X)^{-1} X' A^{-1} r
$$
并且把它写成“每个因子收益是股票收益的加权和”：
$$
\hat b_k = \sum_{n=1}^N c_{kn} r_n
$$
这一步是全章最漂亮的解释：因子收益 = 因子投资组合（factor portfolio）的收益。

### $F$：因子协方差矩阵怎么来？

有了每期 $\hat b(t)$，你就能用时间序列估计因子协方差，并且强调它应该是对你的持有期（horizon）的预测。他们说细节很大（可以用加权历史、Bayesian prior、甚至 Engle 的条件异方差思想）。

你可以把它理解成：
 - 回归阶段解决“当期因子收益是多少”；
 - F 阶段解决“这些因子未来怎么一起波动（风险）”。

 ### $A$：特异风险怎么建模？

因为模型按定义解释不了 u，所以只能建模特异方差。它把特异方差拆成：
$$
u_n^2(t) = S(t)\,[1+v_n(t)]
$$
并约束：
$$
\sum_{n=1}^N v_n(t)=0,\qquad S(t)=\frac{1}{N}\sum_{n=1}^N u_n^2(t)
$$
即 $S(t)$ 是横截面平均特异方差，$v_n(t)$ 描述个股相对平均水平的偏离。

预测上：
 - $S(t)$ 用时间序列模型；
 - $v_n(t)$ 用横截面线性模型（常用一些 risk indices 因子 + 最近的特异收益平方等），并做 pooled regression + outlier trim。

这里你会看到：risk indices 不只解释共同风险，也被拿来解释“谁更容易有大特异波动”（比如小盘、低流动、盈利不稳定公司特异风险更高）。

## 用法：给定 $(X,F,A)$，如何算组合风险、active risk、beta、残差风险？

### 组合暴露与总方差

持仓$h_p$的因子暴露：
$$
x_p = X' h_p
$$
组合方差：
$$
\sigma_p^2 = x_p' F x_p + h_p' A h_p = h_p' V h_p
$$

### 相对基准的 active risk（tracking error）
$$
h_{pA}=h_p-h_B,\quad x_{pA}=X'h_{pA}
$$
$$
W_p^2 = x_{pA}' F x_{pA} + h_{pA}' A h_{pA}
$$
并指出：common-factor 与 specific 两块能拆开，是因为两者不相关。 ￼

### beta 分解：为什么市场 beta 既有因子贡献也有特异贡献？

他们定义相对基准$h_B$的单资产 beta 向量 B（由协方差/方差定义）。然后推出：
$$
B = X b + d
$$
意思是：资产 beta 有一部分来自因子暴露（通常行业占主导），另一部分来自“基准里该股票的特异协方差结构”（如果股票不在基准里，这个 specific contribution 为 0）。 ￼

### 系统性风险 vs 残差风险

并给出把总风险拆为市场项与残差项的表达（并可定义残差协方差矩阵 $V_R$）。 ￼

## 风险归因：边际贡献（marginal contribution）是核心工具

当你想说“风险到底来自哪里”，最大麻烦是协方差项没法唯一分摊。G&K 的做法是：先看边际。
- 总风险的边际贡献（对持仓的偏导）：
$$
\mathrm{MCTR} = \frac{V h_p}{\sigma_p}
$$

- active risk 的边际贡献（类似）：
$$
\mathrm{MCAR} = \frac{V h_{pA}}{W_p}
$$

然后他们把 active risk 的边际贡献再拆成“市场 + 残差”的组合（3A.24–3A.26）。 ￼

因子层面的边际贡献（这和你理解 risk indices 很关键）

“改变单个因子暴露、其他因子暴露不变”并不是随便加一只股票能做到的，所以他们选择加 factor portfolio 来实现“只动一个因子”。

于是定义因子边际贡献（FMCAR），并据此把 active risk 归因到各个因子（包括 risk indices）和特异项。

## 一个收尾例子：为什么“市场越波动，相关性越高”？

他们用单因子模型推导相关系数：

当两只股票 beta=1、残差风险相同，相关系数变成
$$
\rho = \frac{\sigma_M^2}{\sigma_M^2 + w^2}
$$

如果$w$大致不随市场波动变动，那么 $\sigma_M^2$ 越大，$\rho$ 越大：市场波动大 → 相关性上升。 ￼

这段其实在强调：风险模型不是只为“算一个数字”，而是为了给你“相关性从何而来”的结构性解释。